# IAD Argument Mining — Clean Colab Notebook
Goal: prepare datasets (AAE + AbstRCT), train token-classification baselines, evaluate with `argumentation_base/evaluate.py`, and export `final_artifacts.zip`.

**GPU:** Runtime → Change runtime type → GPU (Tesla T4 is fine).

In [1]:
# ==== 0) Environment & paths ====

!pip -q install "transformers==5.0.0" datasets evaluate accelerate

import os, json, glob, zipfile, shutil, random
import numpy as np
import torch
from datasets import Dataset

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available(): torch.cuda.manual_seed_all(SEED)

print("GPU available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device:", torch.cuda.get_device_name(0))

BASE_DIR = "/content/iad_argument_mining"
RAW_DIR  = f"{BASE_DIR}/data/raw"
PROC_DIR = f"{BASE_DIR}/data/processed"
PRED_DIR = f"{BASE_DIR}/predictions"
MODEL_DIR= f"{BASE_DIR}/models"
FINAL_DIR= f"{BASE_DIR}/final_artifacts"
for d in [RAW_DIR, PROC_DIR, PRED_DIR, MODEL_DIR, FINAL_DIR]:
    os.makedirs(d, exist_ok=True)

REPO_URL = "https://github.com/moussisto2/Argument-Mining-with-LLM.git"
REPO_DIR = "/content/Argument-Mining-with-LLM"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.4 MB/s eta 0:00:00
GPU available: True
Device: Tesla T4


## 1) Get project files from GitHub

In [2]:
# ==== 1) Clone repo + copy raw assets ====
if os.path.exists(REPO_DIR):
    shutil.rmtree(REPO_DIR)

!git clone --depth 1 "{REPO_URL}" "{REPO_DIR}"

# Expecting the repo to contain: data/raw/*.zip (datasets + argumentation_base)
repo_raw = os.path.join(REPO_DIR, "data", "raw")
assert os.path.exists(repo_raw), f"Repo missing {repo_raw}"

# Copy everything from repo_raw -> RAW_DIR
for src in glob.glob(os.path.join(repo_raw, "*")):
    dst = os.path.join(RAW_DIR, os.path.basename(src))
    if os.path.isdir(src):
        if os.path.exists(dst): shutil.rmtree(dst)
        shutil.copytree(src, dst)
    else:
        shutil.copy2(src, dst)

print("✅ Copied repo data/raw into:", RAW_DIR)
print("Top-level RAW_DIR:", sorted(os.listdir(RAW_DIR))[:50])

# Unzip all zip files in RAW_DIR
zip_paths = sorted(glob.glob(os.path.join(RAW_DIR, "*.zip")))
print(f"📦 Unzipping {len(zip_paths)} zip files...")
for zp in zip_paths:
    print(" -", os.path.basename(zp))
    with zipfile.ZipFile(zp, "r") as zf:
        zf.extractall(RAW_DIR)

print("✅ After unzip, RAW_DIR top-level:", sorted(os.listdir(RAW_DIR))[:80])


Cloning into '/content/Argument-Mining-with-LLM'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 20 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (20/20), 8.61 MiB | 11.99 MiB/s, done.
Resolving deltas: 100% (1/1), done.
✅ Copied repo data/raw into: /content/iad_argument_mining/data/raw
Top-level RAW_DIR: ['ArgumentAnnotatedEssays-1.0.zip', 'ArgumentAnnotatedEssays-2.0.zip', 'abstrct-master dataset.zip', 'argumentation_base-main.zip', 'brat-project.zip', 'de.tudarmstadt.ukp.dkpro.argumentation.componentclassification.zip']
📦 Unzipping 6 zip files...
 - ArgumentAnnotatedEssays-1.0.zip
 - ArgumentAnnotatedEssays-2.0.zip
 - abstrct-master dataset.zip
 - argumentation_base-main.zip
 - brat-project.zip
 - de.tudarmstadt.ukp.dkpro.argumentation.componentclassification.zip
✅ After unzip, RAW_DIR top-level: ['ArgumentAnnotatedEssays-1.0', 'ArgumentAnnotatedEssa

## 2) Locate datasets + evaluation script
We use `argumentation_base-main/evaluate.py` for the official span-level metrics.

In [3]:
# ==== 2) Sanity check paths ====
ARG_BASE = f"{RAW_DIR}/argumentation_base-main"
AAE_DIR  = f"{RAW_DIR}/ArgumentAnnotatedEssays-2.0"
ABST_DIR = f"{RAW_DIR}/abstrct-master"

EVAL_PY  = f"{ARG_BASE}/evaluate.py"

print("argumentation_base:", os.path.exists(ARG_BASE), ARG_BASE)
print("AAE:", os.path.exists(AAE_DIR), AAE_DIR)
print("AbstRCT:", os.path.exists(ABST_DIR), ABST_DIR)
print("evaluate.py:", os.path.exists(EVAL_PY), EVAL_PY)

assert os.path.exists(EVAL_PY), "evaluate.py missing — check argumentation_base zip in repo data/raw"


argumentation_base: True /content/iad_argument_mining/data/raw/argumentation_base-main
AAE: True /content/iad_argument_mining/data/raw/ArgumentAnnotatedEssays-2.0
AbstRCT: True /content/iad_argument_mining/data/raw/abstrct-master
evaluate.py: True /content/iad_argument_mining/data/raw/argumentation_base-main/evaluate.py


## 3) Convert BRAT → unified JSON (tokens/spans/rels)
Outputs saved under `data/processed/`:
- `aae_{train,dev,test}.json`
- `abstrct_{train,dev,test}.json`

In [4]:
# ==== 3) BRAT parsers (AAE + AbstRCT) ====
import re

def parse_brat_ann(ann_text):
    spans = []
    rels = []
    # T lines: T1\tClaim 10 20\ttext
    for line in ann_text.splitlines():
        line=line.strip()
        if not line:
            continue
        if line.startswith("T"):
            # handle multi-space ranges
            parts = line.split("\t")
            if len(parts) < 3:
                continue
            _, meta, _txt = parts[0], parts[1], parts[2]
            meta_parts = meta.split()
            lbl = meta_parts[0]
            start = int(meta_parts[1]); end = int(meta_parts[2]) - 1  # char end inclusive
            spans.append((lbl, start, end))
        elif line.startswith("R"):
            # R1\tsupports Arg1:T1 Arg2:T2
            parts = line.split("\t")
            if len(parts) < 2:
                continue
            meta = parts[1].split()
            name = meta[0]
            arg1 = meta[1].split(":")[1]
            arg2 = meta[2].split(":")[1]
            rels.append((name, arg1, arg2))
        # Ignore A lines (stance) for this baseline
    return spans, rels

def tokenize_with_offsets(text):
    # simple whitespace tokenizer with char offsets
    tokens=[]
    for m in re.finditer(r"\S+", text):
        tokens.append({"str": m.group(0), "start": m.start(), "end": m.end()-1})
    return tokens

def char_spans_to_bio(tokens, spans):
    # spans: list of (label, char_start, char_end_inclusive)
    tags = ["O"]*len(tokens)
    for lbl, s, e in spans:
        # mark tokens whose char span overlaps [s,e]
        for i,t in enumerate(tokens):
            if t["end"] < s or t["start"] > e:
                continue
            prefix = "B" if tags[i] == "O" else "I"
            # if first token for this span
            # simple rule: first overlap gets B-, rest I-
        # second pass to enforce proper BIO per span
    # Better: for each span, collect overlapping token indices, then assign B/I
    tags = ["O"]*len(tokens)
    for lbl, s, e in spans:
        idxs=[i for i,t in enumerate(tokens) if not (t["end"] < s or t["start"] > e)]
        if not idxs:
            continue
        tags[idxs[0]] = f"B-{lbl}"
        for j in idxs[1:]:
            tags[j] = f"I-{lbl}"
    return tags

def build_doc_from_brat(txt_path, ann_path):
    text = open(txt_path, "r", encoding="utf-8", errors="ignore").read()
    ann  = open(ann_path, "r", encoding="utf-8", errors="ignore").read()
    spans_char, rels_ref = parse_brat_ann(ann)
    toks = tokenize_with_offsets(text)
    bio = char_spans_to_bio(toks, spans_char)

    # Build token dicts with linear idx
    tok_dicts=[]
    for i,(t,tag) in enumerate(zip(toks,bio)):
        tok_dicts.append({"idx": i, "str": t["str"], "arg": tag})

    # Convert char spans to token spans (start/end in token idx)
    token_spans=[]
    for lbl, s, e in spans_char:
        idxs=[i for i,t in enumerate(toks) if not (t["end"] < s or t["start"] > e)]
        if not idxs:
            continue
        token_spans.append({"name": lbl, "start": idxs[0], "end": idxs[-1]})

    # Relations: map T ids to token spans (best-effort by matching order)
    # For this baseline we keep rels empty (token-classification only).
    doc = {"tokens":[tok_dicts], "spans":token_spans, "rels":[]}
    return doc

print("Parser ready.")


Parser ready.


In [5]:
# ==== 3b) Build processed JSONs ====
# AAE: use split CSV from AAE v2.0
import pandas as pd

AAE_BRAT_DIR = os.path.join(AAE_DIR, "brat_project_unzipped", "brat-project-final")
if not os.path.exists(AAE_BRAT_DIR):
    # unzip nested brat zip if needed
    nested = os.path.join(AAE_DIR, "brat-project-final.zip")
    assert os.path.exists(nested), f"Missing {nested}"
    out = os.path.join(AAE_DIR, "brat_project_unzipped")
    os.makedirs(out, exist_ok=True)
    with zipfile.ZipFile(nested) as zf:
        zf.extractall(out)
    # update path
    AAE_BRAT_DIR = os.path.join(out, "brat-project-final")

split_csv = os.path.join(AAE_DIR, "train-test-split.csv")
spl = pd.read_csv(split_csv, sep=";")
train_ids = set(spl[spl["SET"]=="TRAIN"]["ID"].tolist())
test_ids  = set(spl[spl["SET"]=="TEST"]["ID"].tolist())

# fixed small dev split: take first 32 train docs as dev (same as your run)
train_ids_sorted = sorted(train_ids)
dev_ids = set(train_ids_sorted[:32])
train_ids2 = set(train_ids_sorted[32:])

def build_aae_split(ids):
    docs=[]
    for eid in ids:
        txt=os.path.join(AAE_BRAT_DIR, f"{eid}.txt")
        ann=os.path.join(AAE_BRAT_DIR, f"{eid}.ann")
        if os.path.exists(txt) and os.path.exists(ann):
            docs.append(build_doc_from_brat(txt, ann))
    return docs

aae_train = build_aae_split(train_ids2)
aae_dev   = build_aae_split(dev_ids)
aae_test  = build_aae_split(test_ids)

print("AAE sizes:", len(aae_train), len(aae_dev), len(aae_test))

json.dump(aae_train, open(os.path.join(PROC_DIR,"aae_train.json"),"w"))
json.dump(aae_dev,   open(os.path.join(PROC_DIR,"aae_dev.json"),"w"))
json.dump(aae_test,  open(os.path.join(PROC_DIR,"aae_test.json"),"w"))

# AbstRCT: already in BRAT-like .ann/.txt split by folders
ABST_CORPUS = os.path.join(ABST_DIR, "AbstRCT_corpus", "data")
assert os.path.exists(ABST_CORPUS), f"Missing {ABST_CORPUS}"

def build_abstrct_split(split_name):
    docs=[]
    for root,_,files in os.walk(os.path.join(ABST_CORPUS, split_name)):
        for fn in files:
            if fn.endswith(".txt"):
                base=fn[:-4]
                txt=os.path.join(root, base+".txt")
                ann=os.path.join(root, base+".ann")
                if os.path.exists(ann):
                    docs.append(build_doc_from_brat(txt, ann))
    return docs

ab_train = build_abstrct_split("train")
ab_dev   = build_abstrct_split("dev")
ab_test  = build_abstrct_split("test")
print("AbstRCT sizes:", len(ab_train), len(ab_dev), len(ab_test))

json.dump(ab_train, open(os.path.join(PROC_DIR,"abstrct_train.json"),"w"))
json.dump(ab_dev,   open(os.path.join(PROC_DIR,"abstrct_dev.json"),"w"))
json.dump(ab_test,  open(os.path.join(PROC_DIR,"abstrct_test.json"),"w"))

print("✅ Saved processed JSONs into:", PROC_DIR)


AAE sizes: 290 32 80
AbstRCT sizes: 350 50 300
✅ Saved processed JSONs into: /content/iad_argument_mining/data/processed


## 4) Train & evaluate AAE (RoBERTa)
Token classification with labels: `O, B/I-Claim, B/I-Premise`. We reconstruct spans from BIO before running `evaluate.py`.

In [6]:
# ==== 4) AAE: RoBERTa token-classification ====
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification, set_seed
set_seed(SEED)

aae_train_docs = json.load(open(os.path.join(PROC_DIR,"aae_train.json")))
aae_dev_docs   = json.load(open(os.path.join(PROC_DIR,"aae_dev.json")))
print("AAE docs:", len(aae_train_docs), "train |", len(aae_dev_docs), "dev")

LABELS = ["O", "B-Claim", "I-Claim", "B-Premise", "I-Premise"]
label2id = {l:i for i,l in enumerate(LABELS)}
id2label = {i:l for l,i in label2id.items()}

def docs_to_samples(docs):
    samples=[]
    for doc in docs:
        for parag in doc["tokens"]:
            samples.append({
                "tokens":[t["str"] for t in parag],
                "tags":[label2id.get(t["arg"], 0) for t in parag]
            })
    return samples

train_ds = Dataset.from_list(docs_to_samples(aae_train_docs))
dev_ds   = Dataset.from_list(docs_to_samples(aae_dev_docs))

MODEL_NAME="roberta-base"
tok = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
collator = DataCollatorForTokenClassification(tokenizer=tok)

def tokenize_align(examples):
    enc = tok(examples["tokens"], is_split_into_words=True, truncation=True, max_length=256)
    all_labels=[]
    for i in range(len(examples["tokens"])):
        word_ids = enc.word_ids(batch_index=i)
        word_labels = examples["tags"][i]
        labels=[]
        prev=None
        for w in word_ids:
            if w is None: labels.append(-100)
            elif w != prev: labels.append(word_labels[w])
            else:
                lab = word_labels[w]
                if lab == label2id["B-Claim"]: lab = label2id["I-Claim"]
                if lab == label2id["B-Premise"]: lab = label2id["I-Premise"]
                labels.append(lab)
            prev=w
        all_labels.append(labels)
    enc["labels"]=all_labels
    return enc

train_tok = train_ds.map(tokenize_align, batched=True, remove_columns=["tokens","tags"])
dev_tok   = dev_ds.map(tokenize_align, batched=True, remove_columns=["tokens","tags"])

model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(LABELS),
    id2label=id2label,
    label2id=label2id
)

args = TrainingArguments(
    output_dir=os.path.join(MODEL_DIR,"aae_roberta_tokclf"),
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="no",
    logging_steps=50,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    mask = labels != -100
    acc = (preds[mask] == labels[mask]).mean() if mask.sum() else 0.0
    return {"token_acc": float(acc)}

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_tok,
    eval_dataset=dev_tok,
    data_collator=collator,
    compute_metrics=compute_metrics
)
trainer.train()

# ---- Predict dev and build evaluate.py JSON ----
def sanitize_bio(par):
    prev="O"
    for tok_ in par:
        tag=tok_["arg"]
        if tag.startswith("I-"):
            x=tag.split("-",1)[1]
            if prev=="O" or (not prev.endswith(x)):
                tok_["arg"]=f"B-{x}"
        prev=tok_["arg"]

def spans_from_tokens(doc):
    spans=[]
    cur=None; last=None
    for par in doc["tokens"]:
        for t in par:
            idx=t["idx"]; tag=t["arg"]
            if tag.startswith("B-"):
                if cur is not None:
                    spans.append({"name":cur[0],"start":cur[1],"end":last})
                cur=(tag.split('-',1)[1], idx); last=idx
            elif tag.startswith("I-"):
                lbl=tag.split('-',1)[1]
                if cur is None: cur=(lbl, idx)
                elif cur[0]!=lbl:
                    spans.append({"name":cur[0],"start":cur[1],"end":last})
                    cur=(lbl, idx)
                last=idx
            else:
                if cur is not None:
                    spans.append({"name":cur[0],"start":cur[1],"end":last})
                    cur=None
                last=idx
    if cur is not None:
        spans.append({"name":cur[0],"start":cur[1],"end":last})
    return spans

def predict_paragraph(tokens_):
    enc = tok(tokens_, is_split_into_words=True, truncation=True, max_length=256, return_tensors="pt")
    enc = {k:v.to(model.device) for k,v in enc.items()}
    with torch.no_grad():
        logits = model(**enc).logits[0].cpu().numpy()
    pred_ids = logits.argmax(axis=-1)
    word_ids = tok(tokens_, is_split_into_words=True, truncation=True, max_length=256).word_ids()
    word_pred={}
    for i,w in enumerate(word_ids):
        if w is None: continue
        if w not in word_pred: word_pred[w]=pred_ids[i]
    return [id2label.get(word_pred.get(i,0),"O") for i in range(len(tokens_))]

pred_docs=[]
for doc in aae_dev_docs:
    pd={"tokens":[]}
    for par in doc["tokens"]:
        tokens_=[t["str"] for t in par]
        tags=predict_paragraph(tokens_)
        out=[{"idx":t["idx"],"str":t["str"],"arg":tag} for t,tag in zip(par,tags)]
        sanitize_bio(out)
        pd["tokens"].append(out)
    pd["spans"]=spans_from_tokens(pd)
    pred_docs.append(pd)

pred_path=os.path.join(PRED_DIR,"aae_dev_roberta_sanitized.json")
json.dump(pred_docs, open(pred_path,"w"))
print("💾 Saved:", pred_path)

print("\n==== Eval: AAE dev (RoBERTa) ====")
!python "{EVAL_PY}" "{pred_path}" "{os.path.join(PROC_DIR,'aae_dev.json')}"


AAE docs: 290 train | 32 dev


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

RobertaForTokenClassification LOAD REPORT from: roberta-base
Key                             | Status     | 
--------------------------------+------------+-
lm_head.layer_norm.bias         | UNEXPECTED | 
lm_head.dense.weight            | UNEXPECTED | 
roberta.embeddings.position_ids | UNEXPECTED | 
lm_head.layer_norm.weight       | UNEXPECTED | 
lm_head.dense.bias              | UNEXPECTED | 
lm_head.bias                    | UNEXPECTED | 
classifier.bias                 | MISSING    | 
classifier.weight               | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


Epoch,Training Loss,Validation Loss,Token Acc
1,No log,0.752122,0.727731
2,0.944452,0.682100,0.753568


💾 Saved: /content/iad_argument_mining/predictions/aae_dev_roberta_sanitized.json

==== Eval: AAE dev (RoBERTa) ====


********************** SPANS *************************** 
   STRICT EVALUATION
    > Argument mining spans (unlabeled)
      Precision : 0.11170634920634918 
      Recall    : 0.0536265447014286
      F-score   : 0.0718472581083422
    > Argument mining spans (labeled)
      Precision : 0.08194444444444444 
      Recall    : 0.03958482983734532 
      F-score   : 0.05285702410988451

    RELAXED EVALUATION (α = 0.5)
    > Argument mining spans (unlabeled)
      Precision : 0.5251508907758907 
      Recall    : 0.4293691963619886
      F-score   : 0.4687885624969672
    > Argument mining spans (labeled)
      Precision : 0.4321505751193252 
      Recall    : 0.3089653802187665 
      F-score   : 0.3577585622658071



## 5) Train & evaluate AbstRCT (SciBERT)
Same pipeline, but biomedical domain; model = `allenai/scibert_scivocab_uncased`. We ensure token indices are linear (required by `evaluate.py`).

In [13]:
# 5) AbstRCT: SciBERT token-classification


import os, json, torch, sys, subprocess
import numpy as np
from collections import Counter
from datasets import Dataset
import torch.nn as nn
from transformers import (
    AutoTokenizer, AutoModelForTokenClassification,
    TrainingArguments, DataCollatorForTokenClassification,
    Trainer, set_seed
)

set_seed(SEED)

# ------------------------------------------------------------
# 1) Load data
# ------------------------------------------------------------
ab_train_docs = json.load(open(os.path.join(PROC_DIR, "abstrct_train.json")))
ab_dev_docs   = json.load(open(os.path.join(PROC_DIR, "abstrct_dev.json")))
print("AbstRCT docs:", len(ab_train_docs), "train |", len(ab_dev_docs), "dev")

LABELS = ["O", "B-Claim", "I-Claim", "B-Premise", "I-Premise"]
label2id = {l:i for i,l in enumerate(LABELS)}
id2label = {i:l for l,i in label2id.items()}

# ------------------------------------------------------------
# 2) BIO normalization
# ------------------------------------------------------------
def normalize_to_bio(tags):
    out, prev = [], "O"
    for t in tags:
        t = str(t) if t else "O"

        if t == "O":
            out.append("O"); prev = "O"; continue

        # already BIO
        if t.startswith(("B-","I-")):
            typ = t.split("-",1)[1]
            if t.startswith("I-") and (prev=="O" or not prev.endswith(typ)):
                out.append(f"B-{typ}")
            else:
                out.append(t)
            prev = out[-1]
            continue

        # plain "Claim"/"Premise"
        out.append(f"I-{t}" if prev.endswith(t) else f"B-{t}")
        prev = out[-1]

    # keep only supported
    clean=[]
    for x in out:
        if x == "O":
            clean.append("O")
        else:
            typ = x.split("-",1)[1]
            clean.append(x if typ in ("Claim","Premise") else "O")
    return clean

def docs_to_ds(docs):
    rows=[]
    for d in docs:
        par = d["tokens"][0]
        toks = [t["str"] for t in par]
        bio  = normalize_to_bio([t.get("arg","O") for t in par])
        rows.append({"tokens": toks, "tags": [label2id[x] for x in bio]})
    return Dataset.from_list(rows)

train_ds = docs_to_ds(ab_train_docs)
dev_ds   = docs_to_ds(ab_dev_docs)

# ------------------------------------------------------------
# 3) Tokenization + alignment (max_length=512)
# ------------------------------------------------------------
MODEL = "allenai/scibert_scivocab_uncased"
tok = AutoTokenizer.from_pretrained(MODEL, use_fast=True)
collator = DataCollatorForTokenClassification(tok)

MAX_LEN = 512

def tokenize_align(ex):
    enc = tok(ex["tokens"], is_split_into_words=True, truncation=True, max_length=MAX_LEN)
    labs=[]
    for i in range(len(ex["tokens"])):
        wids = enc.word_ids(batch_index=i)
        wl   = ex["tags"][i]
        cur=[]; prev=None
        for w in wids:
            if w is None:
                cur.append(-100)
            elif w != prev:
                cur.append(wl[w])
            else:
                x = wl[w]
                if x == label2id["B-Claim"]:   x = label2id["I-Claim"]
                if x == label2id["B-Premise"]: x = label2id["I-Premise"]
                cur.append(x)
            prev = w
        labs.append(cur)
    enc["labels"] = labs
    return enc

train_tok = train_ds.map(tokenize_align, batched=True, remove_columns=["tokens","tags"])
dev_tok   = dev_ds.map(tokenize_align, batched=True, remove_columns=["tokens","tags"])

# ------------------------------------------------------------
# 4) Class weights (from word-level training tags)
# ------------------------------------------------------------
cnt = Counter()
for x in train_ds:
    cnt.update(x["tags"])

freq = np.array([cnt[i] for i in range(len(LABELS))], dtype=np.float32)
weights = 1.0 / np.maximum(freq, 1.0)
weights = weights / weights.sum() * len(LABELS)
class_weights = torch.tensor(weights, dtype=torch.float)

print("Label counts:", {LABELS[i]: int(freq[i]) for i in range(len(LABELS))})
print("Class weights:", {LABELS[i]: float(class_weights[i]) for i in range(len(LABELS))})

# ------------------------------------------------------------
# 5) Model + WeightedTrainer (boost recall)
# ------------------------------------------------------------
model = AutoModelForTokenClassification.from_pretrained(
    MODEL,
    num_labels=len(LABELS),
    id2label=id2label,
    label2id=label2id
)

class WeightedTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    # ✅ accept new Trainer signature (transformers >= recent versions)
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        loss_fct = nn.CrossEntropyLoss(
            weight=self.class_weights.to(logits.device),
            ignore_index=-100
        )
        loss = loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))
        return (loss, outputs) if return_outputs else loss


args = TrainingArguments(
    output_dir=os.path.join(MODEL_DIR, "abstrct_scibert"),
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=6,          # was 2
    warmup_ratio=0.1,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="no",
    logging_steps=50,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

trainer = WeightedTrainer(
    model=model,
    args=args,
    train_dataset=train_tok,
    eval_dataset=dev_tok,
    data_collator=collator,
    class_weights=class_weights
)
trainer.train()
model.eval()

# ------------------------------------------------------------
# 6) Prediction + spans (linear idx)
# ------------------------------------------------------------
def sanitize(out):
    prev="O"
    for t in out:
        if t["arg"].startswith("I-"):
            typ = t["arg"].split("-",1)[1]
            if prev=="O" or not prev.endswith(typ):
                t["arg"] = f"B-{typ}"
        prev = t["arg"]

def spans_from(doc):
    S=[]; cur=None; last=None
    for p in doc["tokens"]:
        for t in p:
            i,tag = t["idx"], t["arg"]
            if tag.startswith("B-"):
                if cur:
                    S.append({"name":cur[0], "start":cur[1], "end":last})
                cur = (tag.split("-",1)[1], i)
                last = i
            elif tag.startswith("I-"):
                typ = tag.split("-",1)[1]
                if cur is None or cur[0] != typ:
                    if cur:
                        S.append({"name":cur[0], "start":cur[1], "end":last})
                    cur = (typ, i)
                last = i
            else:
                if cur:
                    S.append({"name":cur[0], "start":cur[1], "end":last})
                    cur = None
                last = i
    if cur:
        S.append({"name":cur[0], "start":cur[1], "end":last})
    return S

@torch.no_grad()
def predict(tokens):
    enc = tok(tokens, is_split_into_words=True, truncation=True, max_length=MAX_LEN, return_tensors="pt")
    enc = {k:v.to(model.device) for k,v in enc.items()}
    logits = model(**enc).logits[0].detach().cpu().numpy()
    preds = logits.argmax(-1)

    wids = tok(tokens, is_split_into_words=True, truncation=True, max_length=MAX_LEN).word_ids()
    wp={}
    for i,w in enumerate(wids):
        if w is not None and w not in wp:
            wp[w] = int(preds[i])

    return [id2label.get(wp.get(i,0), "O") for i in range(len(tokens))]

pred_docs=[]
for d in ab_dev_docs:
    par = d["tokens"][0]
    toks = [t["str"] for t in par]
    tags = predict(toks)
    out  = [{"idx":i, "str":s, "arg":a} for i,(s,a) in enumerate(zip(toks,tags))]
    sanitize(out)
    pd = {"tokens":[out]}
    pd["spans"] = spans_from(pd)
    pred_docs.append(pd)

pred_path = os.path.join(PRED_DIR, "abstrct_dev_scibert_linearidx.json")
json.dump(pred_docs, open(pred_path, "w"))
print("💾 Saved:", pred_path)

# ------------------------------------------------------------
# 7) Reference (linear idx)
# ------------------------------------------------------------
ref_docs=[]
for d in ab_dev_docs:
    par = d["tokens"][0]
    toks = [t["str"] for t in par]
    bio  = normalize_to_bio([t.get("arg","O") for t in par])
    out  = [{"idx":i, "str":s, "arg":a} for i,(s,a) in enumerate(zip(toks,bio))]
    sanitize(out)
    ref_docs.append({"tokens":[out], "spans": d.get("spans", []), "rels": d.get("rels", [])})

ref_path = os.path.join(PROC_DIR, "abstrct_dev_linearidx.json")
json.dump(ref_docs, open(ref_path, "w"))
print("💾 Saved ref:", ref_path)

# --------------------------
# 8) Patch evaluate.py
# ---------------------------
with open(EVAL_PY, "r", encoding="utf-8") as f:
    txt = f.read()

# Conservative patch: only safe denominators (keeps original behavior otherwise)
txt = txt.replace("len(pred_spans)", "len(pred_spans) if len(pred_spans)>0 else 1")
txt = txt.replace("len(ref_spans)",  "len(ref_spans)  if len(ref_spans)>0  else 1")
txt = txt.replace("len(pred_rels)",  "len(pred_rels)  if len(pred_rels)>0  else 1")
txt = txt.replace("len(ref_rels)",   "len(ref_rels)   if len(ref_rels)>0   else 1")

EVAL_SAFE = EVAL_PY.replace(".py", "_safe.py")
with open(EVAL_SAFE, "w", encoding="utf-8") as f:
    f.write(txt)

print("\n==== Eval: AbstRCT dev (SciBERT) ====")
print("Pred spans:", sum(len(d["spans"]) for d in pred_docs))
print("Ref  spans:", sum(len(d["spans"]) for d in ref_docs))

cmd = ["python", EVAL_SAFE, pred_path, ref_path]
res = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

print(res.stdout if res.stdout.strip() else "[no stdout]")
print(res.stderr if res.stderr.strip() else "[no stderr]")


AbstRCT docs: 350 train | 50 dev


Map:   0%|          | 0/350 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Label counts: {'O': 52204, 'B-Claim': 666, 'I-Claim': 12031, 'B-Premise': 1537, 'I-Premise': 33287}
Class weights: {'O': 0.04192637652158737, 'B-Claim': 3.2863731384277344, 'I-Claim': 0.1819237321615219, 'B-Premise': 1.4240238666534424, 'I-Premise': 0.06575313210487366}


Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

BertForTokenClassification LOAD REPORT from: allenai/scibert_scivocab_uncased
Key                                        | Status     | 
-------------------------------------------+------------+-
bert.pooler.dense.bias                     | UNEXPECTED | 
cls.predictions.decoder.weight             | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
bert.pooler.dense.weight                   | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.predictions.decoder.bias               | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
classifier.bias                            | MISSING    | 
classifier.weight                          | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored 

Epoch,Training Loss,Validation Loss
1,No log,0.325674
2,0.780589,0.242288
3,0.253843,0.240723
4,0.152444,0.258344
5,0.105219,0.337944
6,0.078821,0.327926


💾 Saved: /content/iad_argument_mining/predictions/abstrct_dev_scibert_linearidx.json
💾 Saved ref: /content/iad_argument_mining/data/processed/abstrct_dev_linearidx.json

==== Eval: AbstRCT dev (SciBERT) ====
Pred spans: 435
Ref  spans: 326


********************** SPANS *************************** 
   STRICT EVALUATION
    > Argument mining spans (unlabeled)
      Precision : 0.49721969696969703 
      Recall    : 0.6472683982683981
      F-score   : 0.5521720347659099
    > Argument mining spans (labeled)
      Precision : 0.4822943722943724 
      Recall    : 0.6224430014430014 
      F-score   : 0.5338200534139288

    RELAXED EVALUATION (α = 0.5)
    > Argument mining spans (unlabeled)
      Precision : 0.5627478278515432 
      Recall    : 0.8162373737373738
      F-score   : 0.6521602897812617
    > Argument mining spans (labeled)
      Precision : 0.5497731934388281 
      Recall    : 0.7914119769119768 
      F-score   : 0.6352944527197725


[no stderr]


## 6) Final packaging
Creates `final_artifacts/` + `final_artifacts.zip` with predictions, summary, and metadata.

In [15]:
# ==== 6) Summarize + display results + package artifacts ====

import os, json, shutil, zipfile, time

print("\n" + "="*60)
print("FINAL RESULTS SUMMARY")
print("="*60)

# ------------------------------------------------------------
# 1) Display AbstRCT results (SciBERT)
# ------------------------------------------------------------
print("\n[AbstRCT – SciBERT Token Classification]")
print("-"*60)

print(f"Predicted spans : {sum(len(d['spans']) for d in pred_docs)}")
print(f"Reference spans : {sum(len(d['spans']) for d in ref_docs)}")

print("""
STRICT EVALUATION
- Unlabeled spans : Precision ≈ 0.50 | Recall ≈ 0.65 | F1 ≈ 0.55
- Labeled spans   : Precision ≈ 0.48 | Recall ≈ 0.62 | F1 ≈ 0.53

RELAXED EVALUATION (α = 0.5)
- Unlabeled spans : Precision ≈ 0.56 | Recall ≈ 0.82 | F1 ≈ 0.65
- Labeled spans   : Precision ≈ 0.55 | Recall ≈ 0.79 | F1 ≈ 0.64
""")

# ------------------------------------------------------------
# 2) Display AAE results (if present)
# ------------------------------------------------------------
aae_path = os.path.join(PRED_DIR, "aae_dev_roberta_sanitized.json")
if os.path.exists(aae_path):
    print("\n[AAE – RoBERTa Token Classification]")
    print("-"*60)
    print("Predictions file found:", aae_path)
    print("⚠ Metrics were computed earlier (see notebook logs).")
else:
    print("\n[AAE]")
    print("-"*60)
    print("No AAE predictions found.")

# ------------------------------------------------------------
# 3) Copy final artifacts
# ------------------------------------------------------------
print("\nPackaging artifacts...")
files_to_copy = [
    (os.path.join(PRED_DIR, "aae_dev_roberta_sanitized.json"),
     os.path.join(FINAL_DIR, "aae_dev_roberta_sanitized.json")),
    (os.path.join(PRED_DIR, "abstrct_dev_scibert_linearidx.json"),
     os.path.join(FINAL_DIR, "abstrct_dev_scibert_linearidx.json")),
]

os.makedirs(FINAL_DIR, exist_ok=True)

for src, dst in files_to_copy:
    if os.path.exists(src):
        shutil.copy2(src, dst)
        print(f"✔ Copied {os.path.basename(src)}")
    else:
        print(f"✘ Missing {os.path.basename(src)} (skipped)")

# ------------------------------------------------------------
# 4) Save results summary JSON
# ------------------------------------------------------------
summary = {
    "AbstRCT": {
        "model": "allenai/scibert_scivocab_uncased",
        "pred_spans": sum(len(d["spans"]) for d in pred_docs),
        "ref_spans": sum(len(d["spans"]) for d in ref_docs),
        "strict": {
            "unlabeled_f1": 0.552,
            "labeled_f1": 0.534
        },
        "relaxed": {
            "unlabeled_f1": 0.652,
            "labeled_f1": 0.635
        }
    }
}

json.dump(
    summary,
    open(os.path.join(FINAL_DIR, "final_results_summary.json"), "w"),
    indent=2
)

# ------------------------------------------------------------
# 5) Save experiment metadata
# ------------------------------------------------------------
meta = {
    "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
    "datasets": {
        "AAE": "ArgumentAnnotatedEssays v2.0",
        "AbstRCT": "AbstRCT corpus"
    },
    "models": {
        "AAE": "roberta-base (token classification)",
        "AbstRCT": "allenai/scibert_scivocab_uncased (token classification)"
    },
    "engineering_notes": [
        "BIO normalization enforced (I → B when needed)",
        "Class-weighted loss to counter O-label dominance",
        "Max sequence length increased to 512",
        "Linear token indices required by evaluate.py",
        "Safe evaluation (division-by-zero protected)"
    ]
}

json.dump(
    meta,
    open(os.path.join(FINAL_DIR, "experiment_metadata.json"), "w"),
    indent=2
)

# ------------------------------------------------------------
# 6) Zip everything
# ------------------------------------------------------------
zip_out = os.path.join(BASE_DIR, "final_artifacts.zip")
if os.path.exists(zip_out):
    os.remove(zip_out)

with zipfile.ZipFile(zip_out, "w", compression=zipfile.ZIP_DEFLATED) as zf:
    for root, _, files in os.walk(FINAL_DIR):
        for fn in files:
            full = os.path.join(root, fn)
            arc  = os.path.relpath(full, os.path.dirname(FINAL_DIR))
            zf.write(full, arc)

print("\n" + "="*60)
print("FINAL ARTIFACTS PACKAGED")
print("="*60)
print("Archive :", zip_out)
print("Contents:", sorted(os.listdir(FINAL_DIR)))
print("✅ DONE")



FINAL RESULTS SUMMARY

[AbstRCT – SciBERT Token Classification]
------------------------------------------------------------
Predicted spans : 435
Reference spans : 326

STRICT EVALUATION
- Unlabeled spans : Precision ≈ 0.50 | Recall ≈ 0.65 | F1 ≈ 0.55
- Labeled spans   : Precision ≈ 0.48 | Recall ≈ 0.62 | F1 ≈ 0.53

RELAXED EVALUATION (α = 0.5)
- Unlabeled spans : Precision ≈ 0.56 | Recall ≈ 0.82 | F1 ≈ 0.65
- Labeled spans   : Precision ≈ 0.55 | Recall ≈ 0.79 | F1 ≈ 0.64


[AAE – RoBERTa Token Classification]
------------------------------------------------------------
Predictions file found: /content/iad_argument_mining/predictions/aae_dev_roberta_sanitized.json
⚠ Metrics were computed earlier (see notebook logs).

Packaging artifacts...
✔ Copied aae_dev_roberta_sanitized.json
✔ Copied abstrct_dev_scibert_linearidx.json

FINAL ARTIFACTS PACKAGED
Archive : /content/iad_argument_mining/final_artifacts.zip
Contents: ['aae_dev_roberta_sanitized.json', 'abstrct_dev_scibert_linearidx.jso